Construir la animación de esfera dura para el caso 3D. Simule N = 50 partículas libres de radio R = 0.1 m contenidas en un volumen de $V = 10 \times 10 \times 10$; al menos 20 segundos. Use un método adecuado para integrar las ecuaciones. Para que se vea fluido, debe reducir el tamaño de la simulación en un factor de 50.

In [2]:
class Particle:
    
    def __init__(self, r0, v0, a0, t, m=1, radius=0.1, Id=0):
        
        self.dt = t[1] - t[0]

        self.r = r0
        self.v = v0
        self.a = a0
        self.K = 100. #dureza
        self.m = m
        self.radius = radius
        self.Id = Id
        self.p = self.m*self.v
        self.f = self.m*self.a
        
        # energia kinetica y Potencial
        self.Ket = 0.5*self.m*np.dot(self.v,self.v)
        self.VEk = np.zeros(len(t))
        self.Pot = 0
        self.POT = np.zeros(len(t))
        
        # Historial
        
        self.R = np.zeros((len(t),len(r0)))
        
        self.V = np.zeros_like(self.R)
        self.A = np.zeros_like(self.R)
        self.F = np.zeros_like(self.R)
        self.P = np.zeros_like(self.R)
        
    #Evolucion de la velocidad, posicion y aceleracion
    def Evolution(self,i):
        
        self.SetPosition(i)
        self.SetVelocity(i)
        self.SetPotencial(i)
      
        self.v += self.dt*self.a
        self.r += self.dt*self.v
        self.Ket = 0.5*self.m*np.dot(self.v,self.v)

    def CalculateForce(self,p):

        d = np.linalg.norm(self.r - p.r)

        compresion = self.radius + p.radius - d
        
        compresion2 = 10 -  np.abs(self.r) + self.radius
        
        if compresion >= 0:
            
            Fn = self.K * compresion**3
            # n es el vector direccion de la fuerza
            self.n = (self.r - p.r)/d
            self.f = np.add(self.f,Fn*self.n)
            self.Pot = (self.K * compresion**4)/4        
          
        elif compresion2[0] <= 0 or compresion2[1] <= 0 or compresion2[2] <= 0:
            if compresion2[0] <= 0:
                Fp = self.K * np.abs(compresion2[0])**3
                if self.r[0] > 0:
                    self.f = np.add(self.f,Fp*np.array([-1,0,0]))
                elif self.r[0] < 0:
                    self.f = np.add(self.f,Fp*np.array([1,0,0]))
                    
                self.Pot = (self.K * compresion2[0]**4)/4
                    
            if compresion2[1] <= 0:
                Fp = self.K * np.abs(compresion2[1])**5
                if self.r[1] > 0:
                    self.f = np.add(self.f,Fp*np.array([0,-1,0]))
                elif self.r[1] < 0:
                    self.f = np.add(self.f,Fp*np.array([0,1,0]))
                    
                self.Pot = (self.K * compresion2[1]**4)/4
                
            if compresion2[2] <= 0:
                Fp = self.K * np.abs(compresion2[2])**5
                if self.r[2] > 0:
                    self.f = np.add(self.f,Fp*np.array([0,0,-1]))
                elif self.r[2] < 0:
                    self.f = np.add(self.f,Fp*np.array([0,0,1]))
                    
                self.Pot = (self.K * compresion2[2]**4)/4
        
        else:
            self.Pot = 0

    def SetPosition(self,i):
        self.R[i] = self.r
    
    def SetVelocity(self,i):
        self.V[i] = self.v
        self.P[i] = self.m*self.v
        self.VEk[i] = self.Ket
        
    def SetPotencial(self,i):
        self.POT[i] = self.Pot
        
    def ResetForce(self):
        self.f[:] = 0.
        self.a[:] = 0.
    
    # Getter
    def GetPosition(self,scale=1):
        return self.R[::scale]
    
    def GetVelocity(self,scale=1):
        return self.V[::scale]
 
    def GetMomentum(self,scale=1):
        return self.P[::scale]
    
    def GetKineticEnergy(self,scale=1):
        return self.VEk[::scale] 
    
    def GetPotentialEnergy(self,scale=1):
        return self.POT[::scale]

def GetParticles(N,t):
    
    Particles = []
    for i in range(N):
        r0 = np.array([np.random.uniform(-10.,10.),np.random.uniform(-10.,10.),np.random.uniform(-10.,10.)])
        v0 = np.array([np.random.uniform(-2.,2.),np.random.uniform(-2.,2.),np.random.uniform(-2.,2.)])
        a0 = np.array([0.,0.,0.])
        
        p0 = Particle(r0,v0,a0,t,Id=i)
        Particles.append(p0)
    return Particles


In [3]:
def RunSimulation(t,Particles):
    for i in tqdm(range(len(t)), desc='Running simulation', unit=' Steps' ):

        for j in range(len(Particles)):
            for k in range(len(Particles)):
                if j!=k:
                    Particles[j].CalculateForce(Particles[k])
        for j in range(len(Particles)):
            Particles[j].Evolution(i)
            Particles[j].ResetForce()
            
            
    return Particles

NameError: name 'np' is not defined

In [4]:
dt = 0.01
tmax = 20
t = np.linspace(0,tmax,dt)
N = 50
Particles = GetParticles(N,t)

NameError: name 'np' is not defined

In [5]:
Particles = RunSimulation(t,Particles)

NameError: name 'RunSimulation' is not defined

In [ ]:
scale = 20
t1 = t[::scale]

fig = plt.figure()
ax = fig.add_subplot(111,projection='3d')

def init():
    
    ax.clear()
    ax.set_xlim(-10,10)
    ax.set_ylim(-10,10)
    ax.set_zlim(-10,10) 

colors=['o','b','r']
    
def Update(i):
    
    init()
    ax.set_title(r't %.3f' %(t1[i]))
    
    for j, p in enumerate(Particles):
        
        x = p.GetPosition(scale)[i,0]
        y = p.GetPosition(scale)[i,1]
        z = p.GetPosition(scale)[i,2]
        
        vx = p.GetVelocity(scale)[i,0]
        vy = p.GetVelocity(scale)[i,1]
        vz = p.GetVelocity(scale)[i,2]
        ax.quiver(x,y,z,vx,vy,vz,color=colors[j])
        ax.scatter(x,y,z,color=colors[j],label=j)
        ax.legend()
    
Animation = anim.FuncAnimation(fig,Update,frames=len(t1),init_func=init)